In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, Callback
import keras

keras.backend.set_image_data_format('channels_last')

# telegram
import telegram
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from pathlib import Path
import os.path
import sys
import datetime
import time

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

In [3]:
class TelegramCallback(Callback):

    def __init__(self, config):
        super(TelegramCallback, self).__init__()
        self.user_id = config['telegram_id']
        self.model_name = config['model_name']
        self.bot = telegram.Bot(config['token'])

    def send_message(self, text):
        try:
            self.bot.send_message(chat_id=self.user_id, text=text)
        except Exception as e:
            print('Message did not send. Error: {}.'.format(e))

    def on_train_begin(self, logs={}):
        text = 'Start training model {} | {}'.format(self.model.name, self.model_name)
        self.send_message(text)

    def on_epoch_end(self, epoch, logs={}):
        text = '{} | Epoch {}.\n'.format(self.model_name, epoch)
        for k, v in logs.items():
            text += '{}: {:.4f}; '.format(k, v)
        self.send_message(text)

In [4]:
pixel_width = {"N5X":1080,"S3Mini":480,"S4":1080,"N6":1440}
pixel_height = {"N5X":1920,"S3Mini":800,"S4":1920,"N6":2560}

def normalizeWidth(row):
    smartphone = row.Phone
    return row.XPress / pixel_width[smartphone]

def normalizeHeight(row):
    smartphone = row.Phone    
    return row.YPress / pixel_height[smartphone]

In [5]:
df = pd.read_pickle("../../data/step03.pkl")
df.Sensor = df.Sensor.apply(lambda x: x[:-22])

In [6]:
df.XPress = df.apply(lambda x: normalizeWidth(x), axis=1)
df.YPress = df.apply(lambda x: normalizeHeight(x), axis=1)

df.Sensor = df.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

np.random.seed(42)
lst = df.PId.unique()
np.random.shuffle(lst)

shape = df.Sensor.iloc[0].shape

dfTrain = df[df.PId.isin(lst[:14])]
dfTest = df[df.PId.isin(lst[14:])]

train_x = np.concatenate(dfTrain.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
train_y = dfTrain[['XPress', 'YPress']].values
train_x = np.concatenate([train_x,train_x,train_x,train_x,train_x,train_x,train_x,train_x])
train_y = np.concatenate([train_y,train_y,train_y,train_y,train_y,train_y,train_y,train_y])

test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
test_y = dfTest[['XPress', 'YPress']].values

In [7]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.2

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
batch_size = 64

tf.get_default_graph()
model = Sequential()

model.add(Conv2D(filters = 36, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (shape)))
model.add(BatchNormalization(axis=-1))        
model.add(MaxPool2D(pool_size=(2,1), padding='Same'))
model.add(Dropout(0.25))

model.add(Flatten()) 

model.add(Dense(64, activation = "relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.5))

model.add(Dense(16, activation = "relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.5))

model.add(Dense(2, activation = "relu"))


## Compiling the model to generate a model
def eucInMM(y_true, y_pred):
    return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
def euc(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))#
def eucStdMM(y_true, y_pred):
    return K.std(K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1))) * constant_pixels
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

optimizer = optimizers.Adam(epsilon=1e-08)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss="mse", optimizer=optimizer, metrics=[rmse, euc])

# Broadcast progress to the tensorboard.
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
file_name = "IMU_general_" + readable_timestamp
tensorboardFolder = "/srv/share/tensorboardfiles/" + file_name

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
#config += "Batch Size: " + str(batch_size) + "\n\n"
#config += "Num of Epochs: " + str(epochs) + "\n\n"

print(file_name)
model.summary()

callbacks = [
    ModelCheckpoint(str(Path.home()) + "/data/models/"+ file_name + ".{epoch:04d}.h5",
                    monitor='val_eucInMM',
                    verbose=0,
                    save_best_only=False,
                    save_weights_only=False
                   ),
    LoggingTensorBoard(settings_str_to_log=config,
                       log_dir=tensorboardFolder,
                       histogram_freq=0,
                       write_graph=True,
                       write_images=True,
                       update_freq='epoch'
                      ),
    ReduceLROnPlateau(monitor='val_loss',
                      patience=10,
                      verbose=1,
                      factor=0.9,
                      min_lr=0.00001),
    TelegramCallback(
        {
            'token': "707020808:AAHJ22G0diUExRP4JUITk1VKuPtBPUVQ-J4",
            'telegram_id': -320890356,
            'model_name': file_name,
        }
    )
]

#history = model.fit_generator(myGenerator("train", batch_size),
#                              steps_per_epoch=len(train_x) // batch_size,
#                              epochs=1000,
#                              verbose=1,
#                              validation_data=myGenerator("test", batch_size),
#                              validation_steps=len(test_x) // batch_size,
#                              callbacks=callbacks
#                             )

history = model.fit(x = train_x, 
                    y = train_y, 
                    batch_size = batch_size,
                   epochs=10000,
                   verbose=2,
                   validation_data=(test_x, test_y),
                   callbacks=callbacks,
                   )
score = model.evaluate(np.expand_dims(test_x,1),np.expand_dims(test_y,1),verbose=2)
print('%s: %.2f' % (model.metrics_names[1], score[1]))
print('Baseline ConvLSTM Error: %.2f' %(1 - score[1]))

IMU_general_20190205_183010
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 78, 6, 36)         360       
_________________________________________________________________
batch_normalization_1 (Batch (None, 78, 6, 36)         144       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 39, 6, 36)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 39, 6, 36)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8424)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                539200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)         

Message did not send. Error: Timed out.
Epoch 48/10000
 - 68s - loss: 0.0227 - rmse: 0.1503 - euc: 0.1888 - val_loss: 0.0306 - val_rmse: 0.1733 - val_euc: 0.2150
Message did not send. Error: Timed out.
Epoch 49/10000
 - 71s - loss: 0.0225 - rmse: 0.1496 - euc: 0.1878 - val_loss: 0.0309 - val_rmse: 0.1741 - val_euc: 0.2161
Epoch 50/10000
 - 69s - loss: 0.0225 - rmse: 0.1497 - euc: 0.1879 - val_loss: 0.0306 - val_rmse: 0.1735 - val_euc: 0.2151
Epoch 51/10000
 - 73s - loss: 0.0224 - rmse: 0.1493 - euc: 0.1875 - val_loss: 0.0305 - val_rmse: 0.1730 - val_euc: 0.2139
Epoch 52/10000
 - 69s - loss: 0.0224 - rmse: 0.1495 - euc: 0.1876 - val_loss: 0.0306 - val_rmse: 0.1733 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 53/10000
 - 69s - loss: 0.0224 - rmse: 0.1492 - euc: 0.1873 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2157
Message did not send. Error: Timed out.
Epoch 54/10000
 - 69s - loss: 0.0224 - rmse: 0.1495 - euc: 0.1877 - val_loss: 0.0307 - val_rmse: 0.1739 - va

 - 68s - loss: 0.0210 - rmse: 0.1446 - euc: 0.1813 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2152
Epoch 105/10000
 - 71s - loss: 0.0209 - rmse: 0.1444 - euc: 0.1810 - val_loss: 0.0309 - val_rmse: 0.1742 - val_euc: 0.2156
Epoch 106/10000
 - 68s - loss: 0.0210 - rmse: 0.1445 - euc: 0.1814 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2154
Epoch 107/10000
 - 68s - loss: 0.0210 - rmse: 0.1445 - euc: 0.1813 - val_loss: 0.0304 - val_rmse: 0.1728 - val_euc: 0.2137

Epoch 00107: ReduceLROnPlateau reducing learning rate to 0.0004304672533180565.
Epoch 108/10000
 - 70s - loss: 0.0209 - rmse: 0.1441 - euc: 0.1808 - val_loss: 0.0307 - val_rmse: 0.1736 - val_euc: 0.2152
Epoch 109/10000
 - 67s - loss: 0.0210 - rmse: 0.1444 - euc: 0.1811 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2146
Message did not send. Error: Timed out.
Epoch 110/10000
 - 68s - loss: 0.0209 - rmse: 0.1444 - euc: 0.1812 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2152
Message did not send. Error: Timed

Epoch 159/10000
 - 69s - loss: 0.0203 - rmse: 0.1421 - euc: 0.1781 - val_loss: 0.0306 - val_rmse: 0.1732 - val_euc: 0.2144
Epoch 160/10000
 - 69s - loss: 0.0203 - rmse: 0.1420 - euc: 0.1780 - val_loss: 0.0307 - val_rmse: 0.1736 - val_euc: 0.2147
Epoch 161/10000
 - 69s - loss: 0.0203 - rmse: 0.1421 - euc: 0.1781 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2152
Epoch 162/10000
 - 68s - loss: 0.0203 - rmse: 0.1421 - euc: 0.1780 - val_loss: 0.0306 - val_rmse: 0.1733 - val_euc: 0.2141
Message did not send. Error: Timed out.
Epoch 163/10000
 - 66s - loss: 0.0203 - rmse: 0.1423 - euc: 0.1783 - val_loss: 0.0306 - val_rmse: 0.1733 - val_euc: 0.2146
Message did not send. Error: Timed out.
Epoch 164/10000
 - 69s - loss: 0.0203 - rmse: 0.1422 - euc: 0.1784 - val_loss: 0.0306 - val_rmse: 0.1734 - val_euc: 0.2148
Message did not send. Error: Timed out.
Epoch 165/10000
 - 67s - loss: 0.0203 - rmse: 0.1421 - euc: 0.1780 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2152
Epoch 166/10000
 - 

Epoch 216/10000
 - 74s - loss: 0.0200 - rmse: 0.1412 - euc: 0.1769 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2147
Epoch 217/10000
 - 75s - loss: 0.0200 - rmse: 0.1409 - euc: 0.1767 - val_loss: 0.0308 - val_rmse: 0.1736 - val_euc: 0.2147

Epoch 00217: ReduceLROnPlateau reducing learning rate to 0.0001350851875031367.
Epoch 218/10000
 - 69s - loss: 0.0200 - rmse: 0.1410 - euc: 0.1766 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2148
Message did not send. Error: Timed out.
Epoch 219/10000
 - 72s - loss: 0.0199 - rmse: 0.1409 - euc: 0.1765 - val_loss: 0.0306 - val_rmse: 0.1732 - val_euc: 0.2142
Epoch 220/10000
 - 71s - loss: 0.0201 - rmse: 0.1413 - euc: 0.1769 - val_loss: 0.0307 - val_rmse: 0.1734 - val_euc: 0.2147
Epoch 221/10000
 - 74s - loss: 0.0200 - rmse: 0.1412 - euc: 0.1768 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2147
Epoch 222/10000
 - 70s - loss: 0.0200 - rmse: 0.1411 - euc: 0.1768 - val_loss: 0.0306 - val_rmse: 0.1733 - val_euc: 0.2142
Message did not se

...

Epoch 804/10000
 - 69s - loss: 0.0196 - rmse: 0.1396 - euc: 0.1748 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2146
Epoch 805/10000
 - 68s - loss: 0.0196 - rmse: 0.1395 - euc: 0.1748 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 806/10000
 - 68s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1747 - val_loss: 0.0308 - val_rmse: 0.1739 - val_euc: 0.2151
Message did not send. Error: Timed out.
Epoch 807/10000
 - 67s - loss: 0.0196 - rmse: 0.1396 - euc: 0.1747 - val_loss: 0.0308 - val_rmse: 0.1739 - val_euc: 0.2154
Epoch 808/10000
 - 69s - loss: 0.0196 - rmse: 0.1397 - euc: 0.1748 - val_loss: 0.0308 - val_rmse: 0.1739 - val_euc: 0.2150
Epoch 809/10000
 - 66s - loss: 0.0195 - rmse: 0.1394 - euc: 0.1746 - val_loss: 0.0307 - val_rmse: 0.1736 - val_euc: 0.2149
Epoch 810/10000
 - 66s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1745 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2147
Epoch 811/10000
 - 68s - loss: 0.0195 - rmse: 0.1395 - euc:

Epoch 865/10000
 - 65s - loss: 0.0196 - rmse: 0.1397 - euc: 0.1749 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2151
Epoch 866/10000
 - 68s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1747 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2148
Message did not send. Error: Timed out.
Epoch 867/10000
 - 66s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1746 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2150
Message did not send. Error: Timed out.
Epoch 868/10000
 - 66s - loss: 0.0196 - rmse: 0.1395 - euc: 0.1746 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 869/10000
 - 66s - loss: 0.0196 - rmse: 0.1398 - euc: 0.1751 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2149
Epoch 870/10000
 - 69s - loss: 0.0196 - rmse: 0.1398 - euc: 0.1751 - val_loss: 0.0308 - val_rmse: 0.1739 - val_euc: 0.2153
Epoch 871/10000
 - 66s - loss: 0.0196 - rmse: 0.1396 - euc: 0.1748 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2149
Epoch 872/10000
 - 

 - 67s - loss: 0.0196 - rmse: 0.1396 - euc: 0.1748 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 928/10000
 - 64s - loss: 0.0195 - rmse: 0.1394 - euc: 0.1745 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 929/10000
 - 66s - loss: 0.0196 - rmse: 0.1397 - euc: 0.1749 - val_loss: 0.0307 - val_rmse: 0.1736 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 930/10000
 - 65s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1745 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2150
Message did not send. Error: Timed out.
Epoch 931/10000
 - 68s - loss: 0.0195 - rmse: 0.1394 - euc: 0.1746 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2149
Epoch 932/10000
 - 65s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1747 - val_loss: 0.0308 - val_rmse: 0.1739 - val_euc: 0.2151
Epoch 933/10000
 - 64s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1747 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.

 - 66s - loss: 0.0195 - rmse: 0.1394 - euc: 0.1745 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 988/10000
 - 64s - loss: 0.0196 - rmse: 0.1397 - euc: 0.1749 - val_loss: 0.0307 - val_rmse: 0.1735 - val_euc: 0.2147
Epoch 989/10000
 - 68s - loss: 0.0195 - rmse: 0.1393 - euc: 0.1744 - val_loss: 0.0306 - val_rmse: 0.1734 - val_euc: 0.2146
Epoch 990/10000
 - 64s - loss: 0.0195 - rmse: 0.1393 - euc: 0.1745 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2150
Epoch 993/10000
 - 67s - loss: 0.0195 - rmse: 0.1394 - euc: 0.1746 - val_loss: 0.0307 - val_rmse: 0.1736 - val_euc: 0.2148
Epoch 994/10000
 - 64s - loss: 0.0195 - rmse: 0.1392 - euc: 0.1742 - val_loss: 0.0308 - val_rmse: 0.1737 - val_euc: 0.2148
Epoch 995/10000
 - 67s - loss: 0.0196 - rmse: 0.1395 - euc: 0.1747 - val_loss: 0.0307 - val_rmse: 0.1737 - val_euc: 0.2149
Message did not send. Error: Timed out.
Epoch 996/10000
 - 65s - loss: 0.0196 - rmse: 0.1397 - euc: 0.1747 - val_lo

 - 66s - loss: 0.0195 - rmse: 0.1395 - euc: 0.1745 - val_loss: 0.0308 - val_rmse: 0.1738 - val_euc: 0.2152
Message did not send. Error: Timed out.
Epoch 1050/10000
